# Miniproject 1
Fingermovements predicting

In [ ]:
# IMPORTS
import numpy as np
import sklearn
import matplotlib as plt
import torch
from torch import Tensor, nn
from torch.autograd import Variable

#### Importing data

In [ ]:
import dlc_bci as bci
train_input , train_target = bci.load(root = './ data_bci')
print ( str ( type ( train_input ) ) , train_input.size() )
print ( str ( type ( train_target ) ) , train_target.size() )
test_input , test_target = bci.load(root = './ data_bci', train = False)
print ( str ( type ( test_input ) ) , test_input.size())
print ( str ( type ( test_target ) ) , test_target.size())

In [ ]:
# flatten the dimensions with 28 features with 50 samples per feature into 1400 "features"
train_input.view(-1, train_input.size()[1]*train_input.size()[2]).size()

### Define linear model

In [ ]:
def DenseNet(input_dim, output_dim, nb_hidden_layers=1, hidden_width=100):
    if nb_hidden_layers < 1:
        print("you need at least one hidden layer")
        model = None
    else:
        layers = []
        layers.append(nn.Linear(input_dim, hidden_width))
        layers.append(nn.ReLU())
        for i in range(nb_hidden_layers-1):
            layers.append(nn.Linear(hidden_width, hidden_width))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_width, output_dim))
        layers.append(nn.Sigmoid())
        
        model = torch.nn.Sequential(*layers)
    return model
    

In [ ]:

class Linear_regression_model(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Linear_regression_model, self).__init__() # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_dim, output_dim) # 1st Full-Connected Layer: 784 (input data) -> 10 (output class
        self.sig = nn.Sigmoid()
        
    def forward(self, x): # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.sig(out)
        return out

### Create different models model and define loss criterion and optimizer

In [ ]:

input_dim = train_input.size()[1]*train_input.size()[2]
output_dim = 1
nb_hidden_layers = 2
hidden_width = 500
# initialize the model
densemodel_1 = DenseNet(input_dim, output_dim)
linear_model = Linear_regression_model(input_dim, output_dim)


### Choose model and define loss criterion and optimizer

In [ ]:

model = densemodel_1
# define loss criterion
criterion = nn.MSELoss()
# define optimizer function and input based on model
learning_rate = 0.0001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

### Training model

In [ ]:
epochs = 100000 #2000

for epoch in range(epochs):
    epoch += 1
    
    inputs = Variable(train_input.view(-1, train_input.size()[1]*train_input.size()[2]))
    labels = Variable(train_target.float())
    
    # clear gradients
    optimizer.zero_grad()
    
    outputs = model.forward(inputs)
    loss = criterion(outputs, labels)
    loss.backward() # "backpropagation" only one layer
    optimizer.step() # updates parameters in model
    preds = outputs.round().abs().view(-1)
    #print(preds)
    #print(labels)
    if epoch % 1000 == 0:
        print("error percantage: ", (int(sum((preds != labels)))/ 316))
        print('epoch {}, loss {}'.format(epoch,loss.data[0]))
    
    # linear model got 0.012 - 0.020 error with lr=0.0001, MSE with 100000 epochs
    # densemodel_1 got 0.005 - 0.009 error with lr=0.0001, MSE with 10000 epochs